In [1]:
import os.path
import os
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid,TUDataset
from torch_geometric.nn import GATConv, GCNConv,GINConv
import numpy as np
import igraph as ig
from torch_geometric.nn import global_mean_pool,global_add_pool

from functools import reduce
import pickle
from sklearn.model_selection import KFold,StratifiedKFold
import torch.optim as optim
import csv
import pandas as pd
from torch_geometric.loader import DataLoader
from torch_geometric.utils import degree
from ogb.graphproppred import PygGraphPropPredDataset
from torch_geometric.data import DataLoader


In [2]:
import utils

In [3]:
dataset_name="ogbg-molhiv"
dataset_name2="ogbg-molhiv"

dataset = PygGraphPropPredDataset(name = dataset_name) 


In [4]:
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')



Dataset: PygGraphPropPredDataset(41127):
Number of graphs: 41127
Number of features: 9
Number of classes: 2


In [5]:
dataset=utils.data_load(dataset,normalize=False)
max_node=utils.max_node_dataset(dataset)
max_node

222

In [6]:
dataset_name="ogbg_molhiv"

### 데이터 체크

In [7]:
file = f'./dataset/{dataset_name}/H1_ver2'

if os.path.isfile(file):
    NEWDATA = torch.load(file)     
    print('file on')

else:        
    SUB_ADJ=[]
    RAW_SUB_ADJ=[]
    NEWDATA=[]
    for i in range(len(dataset)):                    
        
        data=dataset[i]
        v1=data.edge_index[0,:]
        v2=data.edge_index[1,:]
        #print(torch.max(v1))
        adj = torch.zeros((max_node,max_node))
        adj[v1,v2]=1
        adj=adj.numpy()
        (adj==adj.T).all()
        list_feature=(data.x)
        list_adj=(adj)       
        
        #print(dataset[i])
        _, _, _, _, sum_sub_adj = utils.make_cycle_adj_speed_nosl(list_adj,data)
        
        if i % 100 == 0:
            print(i)
            
        #_sub_adj=np.array(sub_adj)

        if len(sum_sub_adj)>0:    
            new_adj=np.stack((list_adj,sum_sub_adj),0)
        else :
            sum_sub_adj=np.zeros((1, list_adj.shape[0], list_adj.shape[1]))
            new_adj=np.concatenate((list_adj.reshape(1, list_adj.shape[0], list_adj.shape[1]),sum_sub_adj),0)

        #SUB_ADJ.append(new_adj)
        SUB_ADJ=new_adj
        #------합치기
        data=dataset[i]
        check1=torch.sum(data.edge_index[0]-np.where(SUB_ADJ[0]==1)[0])+torch.sum(data.edge_index[1]-np.where(SUB_ADJ[0]==1)[1])
        if check1 != 0 :
            print('error')

        data.cycle_index=torch.stack((torch.LongTensor(np.where(SUB_ADJ[1]!=0)[0]), torch.LongTensor(np.where(SUB_ADJ[1]!=0)[1])),1).T.contiguous()
        #data.cycle_attr = torch.FloatTensor(SUB_ADJ[1][np.where(SUB_ADJ[1]!=0)[0],np.where(SUB_ADJ[1]!=0)[1]]) 
        #FloatTensor 형태여야됨 
        NEWDATA.append(data)
        
    torch.save(NEWDATA,file)

file on


In [8]:
dataset_class=[]
for i in range(len(dataset)):
    dataset_class.append(dataset[i].y)

In [2]:
from nets import Cy2C_GCN_OGB_1,Cy2C_GCN_OGB_3
from Trainer_ogb import Trainer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_name=['Cy2C_GCN_OGB_1','Cy2C_GCN_OGB_3']
class_name=[Cy2C_GCN_OGB_1','Cy2C_GCN_OGB_3]

SyntaxError: invalid syntax (968553625.py, line 5)

In [10]:
lr=1e-3

In [11]:
split_idx = dataset.get_idx_split() 

train_dataset=[]
test_dataset=[]
valid_dataset=[]
for i in split_idx["train"]:
    train_dataset.append(NEWDATA[i])
for i in split_idx["test"]:
    test_dataset.append(NEWDATA[i])
for i in split_idx["valid"]:
    valid_dataset.append(NEWDATA[i])

In [12]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

/home/ubuntu/anaconda3/envs/YODECOPY22/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [13]:
for i,CLASS_NAME in enumerate(class_name):
    for hidden_dim in [80]:
        for drop_mid in [0.3]:
            for dropout in [0.05]:
                for decay in [0.00001]:
                    opt_fun=1
                    for n_layer in [3,5,7]:
                        print(n_layer)
                        name=f'{model_name[i]}_{n_layer}_{hidden_dim}_{dropout}({drop_mid})_{decay}'
                        print('=====================================')
                        print('=====',name,'=====',dataset_name,'=====')
                        print('=====================================')
                        trainer=Trainer(name, dataset_name,NEWDATA,device,CLASS_NAME,dataset.num_node_features,dataset[0].y.shape[1], train_loader, valid_loader, test_loader,lr=lr,hidden_dim=hidden_dim,opt_fun=opt_fun,n_layer=n_layer,num_workers=4,eval_metric=dataset.eval_metric,dataset_name2=dataset_name2,dropout=dropout,decay=decay,n_fold=10,drop_mid=drop_mid)
                        trainer.train()

5
===== Cy2C_GCN_OGB_5_80_0.05(0.3)_1e-05 ===== ogbg_molhiv =====
checkpoint
result


KeyboardInterrupt: 